# Bab 13: Loading and Preprocessing Data with TensorFlow

### 1. Pendahuluan

Bab 13 membahas salah satu aspek paling fundamental dan seringkali paling memakan waktu dalam proyek *Machine Learning*: memuat dan memproses data. Proses ini bisa menjadi sangat rumit, terutama ketika berhadapan dengan dataset yang sangat besar yang tidak muat dalam memori, atau ketika memerlukan langkah pra-pemrosesan yang kompleks.

Bab ini memperkenalkan **Data API (`tf.data`)** dari TensorFlow, sebuah *toolset* yang kuat dan efisien untuk membuat *pipeline* input data. Dengan Data API, Anda dapat dengan mudah membaca data dari berbagai sumber, melakukan transformasi yang kompleks, dan mengalirkannya ke model Anda secara efisien.

Topik utama yang dibahas:
* **Data API (`tf.data`):** Cara membuat *pipeline* data yang efisien, termasuk *chaining transformations*, *shuffling*, *prefetching*, dan paralelisasi.
* **Format TFRecord:** Format biner portabel dari TensorFlow yang sangat efisien untuk menyimpan dan mengakses data dalam jumlah besar.
* **Preprocessing Data:** Cara membuat *preprocessing layer* menggunakan Keras atau `tf.io` untuk menangani fitur numerik dan kategorikal.
* **Proyek TF-Transform dan TFDS:** Pengenalan singkat tentang *tool* tingkat tinggi untuk manajemen data dan *preprocessing*.

---

### 2. Data API (`tf.data`)

Data API berpusat pada konsep **dataset**, yang merepresentasikan urutan *item*. Biasanya, setiap *item* adalah satu *instance* (pasangan fitur dan label), tetapi bisa juga berupa apa saja. TensorFlow menyediakan berbagai cara untuk membuat dataset.

#### a. Membuat Pipeline Data
Cara paling sederhana untuk membuat dataset adalah dengan `tf.data.Dataset.from_tensor_slices()`. Fungsi ini mengambil sebuah tensor dan membuat dataset yang *item*-nya adalah semua *slice* dari tensor tersebut.

Setelah dataset dibuat, Anda dapat menerapkan serangkaian transformasi dengan menyambungkannya (*chaining*).

In [1]:
import tensorflow as tf
from tensorflow import keras

# Membuat dataset dari tensor
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)

# Chaining transformations
dataset = dataset.repeat(3).batch(7)

# Iterasi melalui dataset
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


Output:

`tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)`
`tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)`

### b. Transformasi Umum
map(): Menerapkan fungsi kustom pada setiap item.
filter(): Menyaring item berdasarkan predikat.
shuffle(): Mengacak item dalam dataset. Penting untuk performa Gradient Descent.
batch(): Mengelompokkan item ke dalam batch.
repeat(): Mengulang dataset beberapa kali.
prefetch(): Mempersiapkan batch data berikutnya saat model sedang berlatih dengan batch saat ini, yang dapat secara signifikan meningkatkan performa.

---

### 3. Format File TFRecord
Untuk dataset yang sangat besar, seringkali tidak efisien untuk memiliki ribuan atau jutaan file kecil. Format TFRecord adalah solusi TensorFlow untuk ini. Ini adalah format biner sederhana yang memungkinkan Anda menyimpan urutan record biner. Ini sangat efisien untuk dibaca dan sangat cocok untuk data yang besar.

Setiap record di dalam file TFRecord adalah string byte, dan untuk membuatnya portabel, kita perlu melakukan serialisasi data terlebih dahulu. Format serialisasi standar yang digunakan adalah **protocol buffers**.

In [2]:
# Contoh menulis dan membaca file TFRecord
# (Ini adalah contoh konseptual, implementasi penuh memerlukan lebih banyak detail)

# Menulis ke TFRecord
# with tf.io.TFRecordWriter("my_data.tfrecord") as f:
#     f.write(b"Contoh record pertama")
#     f.write(b"Contoh record kedua")

# Membaca dari TFRecord dengan tf.data API
# filepath = ["my_data.tfrecord"]
# dataset = tf.data.TFRecordDataset(filepath)

### a. Protocol Buffers dan tf.train.Example
`tf.train.Example` adalah protocol buffer yang fleksibel untuk merepresentasikan satu instance data. Ia berisi daftar fitur, di mana setiap fitur bisa berupa ByteList, FloatList, atau Int64List.

### b. Memuat dan Parsing TFRecord
Untuk memuat data dari file TFRecord, kita menggunakan `tf.data.TFRecordDataset`. Setelah dimuat, setiap record (yang masih dalam bentuk string byte terserialisasi) perlu di-parsing menggunakan `tf.io.parse_single_example()`.

---

### 4. Preprocessing Fitur Input
Sebelum memasukkan data ke model, fitur seringkali perlu di-preprocess.

### a. Preprocessing dengan Keras Preprocessing Layers
Keras menyediakan serangkaian layer untuk preprocessing yang dapat dimasukkan langsung ke dalam model Anda. Ini sangat praktis karena layer ini akan secara otomatis diterapkan pada data saat pelatihan dan inferensi.

* **Normalisasi**: `keras.layers.Normalization` dapat menormalkan fitur numerik. Layer ini akan menghitung rata-rata dan standar deviasi dari data pelatihan saat metode `.adapt()` dipanggil.
* **Diskretisasi** (Binning): `keras.layers.Discretization` mengubah fitur numerik berkelanjutan menjadi fitur kategorikal dengan membaginya ke dalam beberapa bin.
* **Fitur Kategorikal**:
    * `keras.layers.StringLookup` atau `IntegerLookup`: Mengubah fitur kategorikal (string atau integer) menjadi indeks integer.
    * `keras.layers.Embedding`: Mengubah indeks integer menjadi embedding vector padat.
    * `keras.layers.CategoryEncoding`: Melakukan one-hot encoding pada fitur kategorikal.
* **Preprocessing Gambar**: Keras juga menyediakan layer seperti Resizing, Rescaling, RandomFlip, dan RandomRotation untuk augmentasi data gambar.

In [3]:
# Contoh pipeline preprocessing untuk data numerik dan kategorikal
# (Asumsikan sudah ada data latih)

# Buat layer untuk normalisasi fitur numerik
# norm_layer = keras.layers.Normalization()
# norm_layer.adapt(X_train_num)

# Buat layer untuk one-hot encoding fitur kategorikal
# cat_layer = keras.layers.CategoryEncoding(num_tokens=num_cat_classes)

# ... kemudian gabungkan dengan model menggunakan Functional API

---

### 5. Dataset TensorFlow (TFDS)
TensorFlow Datasets (TFDS) adalah library yang menyediakan akses mudah ke puluhan dataset machine learning yang siap pakai. Ini menangani semua proses pengunduhan, pemisahan, dan pembuatan `tf.data.Dataset`.

In [5]:
%pip install tensorflow-datasets
import tensorflow_datasets as tfds

# Memuat dataset (misalnya, MNIST)
# as_supervised=True akan mengembalikan tuple (input, label)
dataset, info = tfds.load("mnist", split="train", as_supervised=True, with_info=True)

# Dataset yang dikembalikan sudah berupa objek tf.data.Dataset
# dataset = dataset.map(preprocess_function).shuffle(buffer_size).batch(batch_size)
# model.fit(dataset, epochs=10)


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.3 MB ? eta -:--:--
   - -------------------------------------- 0.3/5.3 MB ? eta -:--:--
   - -------------------------------------- 0.3/5.3 MB ? eta -:--:--
   - -------------------------------------- 0.3/5.3 MB ? eta -:--:--
   --- ------------------------------------ 0.5/5.3 MB 465.6 kB/s eta 0:00:11
   --- ------------------------------------ 0.5/5.3 MB 465.6 kB/s eta 0:00:11
   ----- ---------------------------------- 0.8/5.3 MB 550.1 kB/s eta 0:00:09
   ----- ---------------------------------- 0.8/5.3 MB 550.1 kB/s eta 0:00:09
   ------- -------------------------------- 1.0/5.3 MB 592.2 kB/s eta 0:00:08
   ------- -------------------------------- 1.0/5.3 MB 592.2 kB/s eta 0:00:08
   --------- ------------------------------ 1.3/5.3 MB 610.0 kB/s eta 0:00:07
   ---

c:\Users\Khalif Prawira\AppData\Local\Programs\Orange\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:04<00:01,  1.34s/ url]

Dl Completed...: 100%|██████████| 4/4 [00:09<00:00,  2.70s/ url]

Dl Completed...: 100%|██████████| 4/4 [00:10<00:00,  2.50s/ url]
                                                                        

Dataset mnist downloaded and prepared to C:\Users\Khalif Prawira\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


Menggunakan TFDS dapat secara signifikan menyederhanakan dan mempercepat alur kerja Anda, memungkinkan Anda untuk fokus pada pembangunan dan pelatihan model.